# EDA

In [ ]:
import duckdb
import pandas as pd
from pathlib import Path

duckdb_path = "../data/sakila.duckdb"
sql_load_path = "../sql/load_sakila.sql"

Path(duckdb_path).unlink(missing_ok=True)

with duckdb.connect(duckdb_path) as conn, open(sql_load_path, "r") as ingest_script:
    conn.execute(ingest_script.read())

    print("Loading tables:")
    actor = conn.sql("SELECT * FROM actor").df()
# test test test
print(f"First actor: {actor.iloc[0]['first_name']} {actor.iloc[0]['last_name']}")
print(f"Second actor: {actor.iloc[1]['first_name']} {actor.iloc[1]['last_name']}")
print("DATA LOAD SUCCESSFUL")

Loading tables
First actor: PENELOPE GUINESS
Second actor: NICK WAHLBERG
DATA LOAD SUCCESSFUL
